In [2]:
import logging
#logging.getLogger('tensorflow').disabled = True

import pandas as pd
import requests
import json
import re
import sys
import numpy as np
import nltk
from importlib import reload

sys.path.append('../src')

In [2]:
!pip install bert-tensorflow

In [2]:
pred_sentences = [
    ("Over the whole area, the average level of damage rose to 30%, but further north it rose to 40% and in the south it was rarely above 15%.","30","damage"),
    ("Over the whole area, the average level of damage rose to 30%, but further north it rose to 40% and in the south it was rarely above 15%.","40","damage"),
    ("Over the whole area, the average level of damage rose to 30%, but further north it rose to 40% and in the south it was rarely above 15%.","15","damage"),
    ("Over the whole area, the average level of damage rose to 30%, but further north it rose to 40% and in the south it was rarely above 15%.","30","whole area"),
    ("Over the whole area, the average level of damage rose to 30%, but further north it rose to 40% and in the south it was rarely above 15%.","15","average"),
    ("Productivity was increased by 35 % with plastic bags.", "35", "plastic bag"),
    ("30% respondents were asked about jute sacks", "30", "jute sack"),
    ("It took us 5 months to get used to plastic bags", "5","plastic bag"),
    ("At 120 days, relative to controls, ACK had reduced the final weevil population density by 76 to 82%, floating seeds by 86 to 98% and damaged sunken seeds by 36 to 53%, but increased undamaged sunken seeds by 358 to 572%.","120","ACK")
]

In [ ]:
from interventions_labeling_lib import measurements_labeler
measurements_labeler = reload(measurements_labeler)

_measurements_labeler = measurements_labeler.MeasurementsLabeler("bert_results_5")

W0511 18:05:57.129045 140611470010176 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Started BERT
Started tokenizer loading
Instructions for updating:
Colocations handled automatically by placer.


W0511 18:05:58.094728 140611470010176 deprecation.py:323] From /home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0511 18:05:59.268776 140611470010176 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [4]:
train, test = _measurements_labeler.prepare_datasets("../tmp/measurements_data/train.xlsx", all_dataset=True)

In [5]:
train.columns

Index(['Sentence', 'Number', 'Word Expression', 'Label'], dtype='object')

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

y_train = train["Label"].values
y_train_array = train["Label"].values
x_train = train.drop(["Label"], axis=1).values
skfolds = StratifiedKFold(n_splits=10, random_state=42)
scores = []
ind = 0
for train_index, test_index in skfolds.split(x_train,y_train):
    print("Started %d cv block processing"%ind)
    x_train_folds = x_train[train_index]
    y_train_folds = y_train_array[train_index]
    x_test_folds = x_train[test_index]
    y_test_folds = y_train_array[test_index]
    train_fold = pd.DataFrame(x_train_folds, columns=['Sentence', 'Number', 'Word Expression'])
    train_fold["Label"] = y_train_folds
    test_fold = pd.DataFrame(x_test_folds, columns=['Sentence', 'Number', 'Word Expression'])
    test_fold["Label"] = y_test_folds
    _measurements_labeler = measurements_labeler.MeasurementsLabeler("bert_results_cv_%d"%ind)
    _measurements_labeler.train_model(train_fold, train_fold)
    res_prob, res_label, res_y = _measurements_labeler.evaluate_model(test_fold)
    scores.append(f1_score(res_y, res_label, average="macro"))
    ind += 1

Started 0 cv block processing
Started BERT
Started tokenizer loading
Tokenizrer loaded
Config is done
Beginning Training!


/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Training took time  0:12:21.052292
[[2848   42]
 [  45 1654]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      2890
           1       0.98      0.97      0.97      1699

   micro avg       0.98      0.98      0.98      4589
   macro avg       0.98      0.98      0.98      4589
weighted avg       0.98      0.98      0.98      4589

F1 score:  0.979664992384107
[[282  40]
 [ 35 154]]
              precision    recall  f1-score   support

           0       0.89      0.88      0.88       322
           1       0.79      0.81      0.80       189

   micro avg       0.85      0.85      0.85       511
   macro avg       0.84      0.85      0.84       511
weighted avg       0.85      0.85      0.85       5

Tokenizrer loaded
Config is done
Beginning Training!
Training took time  0:12:17.662186
[[2830   61]
 [  45 1655]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2891
           1       0.96      0.97      0.97      1700

   micro avg       0.98      0.98      0.98      4591
   macro avg       0.97      0.98      0.98      4591
weighted avg       0.98      0.98      0.98      4591

F1 score:  0.9752929634350115
[[295  26]
 [ 34 154]]
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       321
           1       0.86      0.82      0.84       188

   micro avg       0.88      0.88      0.88       509
   macro avg       0.88      0.87      0.87       509
weighted avg       0.88      0.88      0.88       509

F1 score:  0.872324414715719


In [9]:
scores, np.average(scores), np.std(scores), (np.average(scores) - 2*np.std(scores), np.average(scores) + 2*np.std(scores))

([0.8434033268365633,
  0.8831961859979103,
  0.888495159048063,
  0.8372768644309728,
  0.8561330147351653,
  0.7767676767676767,
  0.8100038110572043,
  0.8781086078719951,
  0.8117048028763223,
  0.872324414715719],
 0.8457413864337593,
 0.03516131117115266,
 (0.7754187640914539, 0.9160640087760646))

In [3]:
from interventions_labeling_lib import measurements_labeler
measurements_labeler = reload(measurements_labeler)

_measurements_labeler = measurements_labeler.MeasurementsLabeler("../model/bert_measurements_model")

W0625 14:11:10.933947 139819477702464 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Started BERT
Started tokenizer loading
Used gpu 0
Instructions for updating:
Colocations handled automatically by placer.


W0625 14:11:11.897800 139819477702464 deprecation.py:323] From /home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0625 14:11:13.071826 139819477702464 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': 'bert_results_6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f29694c5710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0625 14:11:14.015396 139819477702464 estimator.py:201] Using config: {'_model_dir': 'bert_results_6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f29694c5710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done


In [10]:
train, test = _measurements_labeler.prepare_datasets("../tmp/measurements_data/test.xlsx", all_dataset=True)
res = _measurements_labeler.evaluate_model(test)

INFO:tensorflow:Writing example 0 of 300


I0625 14:25:43.484949 139819477702464 base_bert_model.py:424] Writing example 0 of 300


INFO:tensorflow:*** Example ***


I0625 14:25:43.486989 139819477702464 base_bert_model.py:400] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0625 14:25:46.641094 139819477702464 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0625 14:25:46.705993 139819477702464 session_manager.py:493] Done running local_init_op.


[[150  42]
 [ 18  90]]
              precision    recall  f1-score   support

           0       0.89      0.78      0.83       192
           1       0.68      0.83      0.75       108

   micro avg       0.80      0.80      0.80       300
   macro avg       0.79      0.81      0.79       300
weighted avg       0.82      0.80      0.80       300

F1 score:  0.7916666666666666


In [4]:
_measurements_labeler.getPredictions(pred_sentences)

INFO:tensorflow:Writing example 0 of 9


[1, 1, 1, 0, 0, 1, 1, 0, 0]

# Experiments with BERT classification for interventions

In [2]:
from utilities import excel_reader
interventions_df = excel_reader.ExcelReader().read_df_from_excel("../hyponyms_for_train.xlsx")
test_interventions_df = excel_reader.ExcelReader().read_df_from_excel("../hyponyms_test_with_predicted_labels.xlsx")

In [3]:
class2id = {
    "Technology intervention": 0,
    "Socioeconomic intervention":    1,
    "Ecosystem intervention":        2,
    "Mechanisation intervention":    3,
    "Storage intervention":          4,
    "Non-intervention": 5
}
id2class = {value: item for item, value in class2id.items()}

In [4]:
interventions_df["label"] = interventions_df["Label"].map(lambda x: class2id[x])
test_interventions_df["label"] = test_interventions_df["Label"].map(lambda x: class2id[x])

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

In [6]:
from bert_models import base_bert_model_interventions
inteventions_labels_model = base_bert_model_interventions.BaseBertModelInterventions("interventions_model_1", list(range(6)), gpu_device_num_hub=1, gpu_device_num =1, 
                                                         batch_size = 64, max_seq_length = 64, label_column="label")

W0611 11:39:03.442067 140031605708608 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Started tokenizer loading
Used gpu 0
Instructions for updating:
Colocations handled automatically by placer.


W0611 11:39:04.198165 140031605708608 deprecation.py:323] From /home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0611 11:39:06.106757 140031605708608 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': 'interventions_model_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5b0d66f358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0611 11:39:07.088077 140031605708608 estimator.py:201] Using config: {'_model_dir': 'interventions_model_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5b0d66f358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done


In [5]:
from sklearn.utils import shuffle
interventions_df = shuffle(interventions_df)

In [8]:
inteventions_labels_model.train_model(interventions_df, interventions_df)

INFO:tensorflow:Writing example 0 of 2008


I0611 10:53:19.197277 140029104518976 base_bert_model.py:424] Writing example 0 of 2008


INFO:tensorflow:*** Example ***


I0611 10:53:19.198580 140029104518976 base_bert_model.py:400] *** Example ***


INFO:tensorflow:guid: None


...

I0611 10:54:42.640921 140029104518976 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0611 10:54:42.741391 140029104518976 session_manager.py:493] Done running local_init_op.


[[652   5   2   5   1  20]
 [  7 214   4   3   0   9]
 [  8   1 124   0   6   3]
 [  6   0   0 114   1   5]
 [  3   3   6   2 107   3]
 [ 24   6   5   3   1 655]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       685
           1       0.93      0.90      0.92       237
           2       0.88      0.87      0.88       142
           3       0.90      0.90      0.90       126
           4       0.92      0.86      0.89       124
           5       0.94      0.94      0.94       694

   micro avg       0.93      0.93      0.93      2008
   macro avg       0.92      0.91      0.91      2008
weighted avg       0.93      0.93      0.93      2008

F1 score:  0.9120455428170927


([array([0.05809062, 0.01410394, 0.01226282, 0.01672174, 0.01141194,
         0.8874089 ], dtype=float32),
  array([0.11364584, 0.04742139, 0.14481889, 0.03429716, 0.05538703,
         0.6044296 ], dtype=float32),
  array([0.11917907, 0.01421885, 0.01893297, 0.7572563 , 0.05234072,
         0.03807202], dtype=float32),
  array([0.04358981, 0.01440416, 0.00664271, 0.00744928, 0.00554451,
         0.92236954], dtype=float32),
  array([0.07594933, 0.06497489, 0.71691483, 0.0350449 , 0.08880764,
         0.01830838], dtype=float32),
  array([0.04156075, 0.01204858, 0.02045236, 0.85479945, 0.04634505,
         0.0247938 ], dtype=float32),
  array([0.02220665, 0.01268258, 0.04032531, 0.1317339 , 0.7776963 ,
         0.0153552 ], dtype=float32),
  array([0.02324919, 0.84801865, 0.07866998, 0.00635613, 0.01167532,
         0.03203079], dtype=float32),
  array([0.0528361 , 0.01187389, 0.00912296, 0.01268792, 0.00875162,
         0.9047275 ], dtype=float32),
  array([0.9254624 , 0.00597457, 0.01

In [9]:
res = inteventions_labels_model.evaluate_model(test_interventions_df)

INFO:tensorflow:Writing example 0 of 400


I0611 10:54:47.032526 140029104518976 base_bert_model.py:424] Writing example 0 of 400


INFO:tensorflow:*** Example ***


I0611 10:54:47.033796 140029104518976 base_bert_model.py:400] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0611 10:54:52.166684 140029104518976 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0611 10:54:52.266193 140029104518976 session_manager.py:493] Done running local_init_op.


[[112   2  10   2   1   8]
 [  3  35   2   0   0   7]
 [  8   0  18   0   0   2]
 [  5   0   0  21   1   0]
 [  4   0   0   0  20   0]
 [ 43   5   1   6   3  81]]
              precision    recall  f1-score   support

           0       0.64      0.83      0.72       135
           1       0.83      0.74      0.79        47
           2       0.58      0.64      0.61        28
           3       0.72      0.78      0.75        27
           4       0.80      0.83      0.82        24
           5       0.83      0.58      0.68       139

   micro avg       0.72      0.72      0.72       400
   macro avg       0.73      0.74      0.73       400
weighted avg       0.74      0.72      0.72       400

F1 score:  0.7281896375048352


In [12]:
for i in range(45, 60):
    print(res[0][i], res[1][i], id2class[res[1][i]], test_interventions_df["Narrow concept"].values[i])

[0.01571067 0.8379847  0.05253926 0.00792936 0.00825082 0.07758525] 1 Socioeconomic intervention private sector
[0.01771423 0.8335716  0.09434523 0.00740228 0.01137348 0.03559314] 1 Socioeconomic intervention health policy highlight role worker education
[0.01284049 0.85629463 0.07896289 0.00879947 0.01383512 0.02926752] 1 Socioeconomic intervention private sector farmer trainer
[0.01270589 0.82621646 0.11102252 0.00902549 0.01552711 0.02550257] 1 Socioeconomic intervention social technology benefit family farmer
[0.18822783 0.5615814  0.1188187  0.02100698 0.03369654 0.07666864] 1 Socioeconomic intervention hired human labour
[0.12892666 0.5602191  0.0760889  0.01462096 0.01289417 0.2072502 ] 1 Socioeconomic intervention PDO scheme # provision DO
[0.4825499  0.10342997 0.34748745 0.01835041 0.0313118  0.01687042] 0 Technology intervention improving performance participatory irrigation management reform
[0.02593041 0.67483234 0.04948941 0.0087673  0.00661365 0.23436691] 1 Socioeconomic

In [6]:
from sklearn.svm import SVC 
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import f1_score

def cross_validation(clf, x_train,y_train,kfolds=10):
    y_train_array = np.asarray(y_train)
    skfolds = StratifiedKFold(n_splits=kfolds, random_state=42)
    scores = []
    for train_index, test_index in skfolds.split(x_train,y_train):
        clone_clf = clone(clf)
        x_train_folds = x_train[train_index]
        y_train_folds = y_train_array[train_index]
        x_test_folds = x_train[test_index]
        y_test_folds = y_train_array[test_index]

        clone_clf.fit(x_train_folds, y_train_folds)
        y_pred = clone_clf.predict(x_test_folds)
        scores.append(f1_score(y_test_folds, y_pred, average="macro"))
    return np.asarray(scores)

def train_svm(train_x, train_y, test_x, test_y):
    svclassifier = SVC(kernel='rbf', probability=True, class_weight={0:2, 1:3,2:4, 3:4, 4:4, 5:3 })  
    svclassifier.fit(train_x, train_y)

    scores = cross_validation(svclassifier, np.concatenate((train_x,test_x),axis=0), train_y+test_y)
    print(scores)
    print("Cross validation score(F1): %f"%(scores.mean()))
    print("Test accuracy : %f" %(svclassifier.score(test_x,test_y)))
    return svclassifier

In [8]:
import gensim
import os
google_model = gensim.models.Word2Vec.load(os.path.join("../model/synonyms_retrained_new","google_plus_our_dataset/", "google_plus_our_dataset.model"))
fast_text_model = gensim.models.Word2Vec.load(os.path.join("../model/synonyms_retrained_new","fast_text_our_dataset/", "fast_text_our_dataset.model"))

In [9]:
from gensim.models.phrases import Phrases, Phraser 
phrases = Phraser.load(os.path.join("../model/synonyms_retrained_new","phrases_3gram.model"))

In [13]:
fast_text_model.most_similar("CSA")

/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('serogroups O26', 0.4490981101989746),
 ('serogroups', 0.4481642246246338),
 ('O157 serogroups', 0.43964269757270813),
 ('serological assay', 0.4087376892566681),
 ('mitis', 0.4040156304836273),
 ('serovar enteritidis', 0.4038276970386505),
 ('serotype enteritidis', 0.40310582518577576),
 ('antigen ELISA', 0.4030936360359192),
 ('serovars', 0.39805352687835693),
 ('viral antigen', 0.3973124027252197)]

In [10]:
from text_processing import text_normalizer
def get_word_expression_embedding(sentence, filter_apply = False):
    word_expressions = set()
    for sent in sentence.split(";"):
        for phr in phrases[text_normalizer.get_stemmed_words_inverted_index(text_normalizer.normalize_text(sent.strip()))]:
            if not filter_apply:
                word_expressions.add(phr.replace("_", " "))
    vec = np.zeros(300)
    for word in word_expressions:
        #if word in google_model.wv:
        #    vec += google_model.wv[word]
        if word in fast_text_model.wv:
            vec += fast_text_model.wv[word]
        else:
            print("%s is not found (%s)"%(word, sentence))
    return vec if len(word_expressions) == 0 else vec/(len(word_expressions))

In [11]:
train_data = interventions_df.values
test_data = test_interventions_df.values

In [31]:
from sklearn.metrics import classification_report, confusion_matrix
train_x, train_y = [np.concatenate((get_word_expression_embedding(data[0]), get_word_expression_embedding(data[0]+";"+data[1]))) for data in train_data], [data[-1] for data in train_data]
test_x, test_y = [np.concatenate((get_word_expression_embedding(data[0]), get_word_expression_embedding(data[0]+";"+data[1]))) for data in test_data], [data[-1] for data in test_data]
svclassifier = train_svm(train_x, train_y, test_x, test_y)
y_pred = svclassifier.predict(test_x)
print(confusion_matrix(test_y,y_pred))  
print(classification_report(test_y,y_pred))

/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warni

[0.6868582  0.69481831 0.70105906 0.68207892 0.70524057 0.73227655
 0.63851446 0.62117397 0.83326245 0.78933149]
Cross validation score(F1): 0.708461
Test accuracy : 0.822500
[[112   2   1   3   0  17]
 [  1  41   0   0   0   5]
 [  2   0  25   0   0   1]
 [  3   0   0  22   0   2]
 [  0   0   0   0  24   0]
 [ 28   1   1   4   0 105]]
              precision    recall  f1-score   support

           0       0.77      0.83      0.80       135
           1       0.93      0.87      0.90        47
           2       0.93      0.89      0.91        28
           3       0.76      0.81      0.79        27
           4       1.00      1.00      1.00        24
           5       0.81      0.76      0.78       139

   micro avg       0.82      0.82      0.82       400
   macro avg       0.87      0.86      0.86       400
weighted avg       0.83      0.82      0.82       400



In [12]:
from sklearn.metrics import classification_report, confusion_matrix
train_x, train_y = [np.concatenate((get_word_expression_embedding(data[0]), get_word_expression_embedding(data[0]+";"+data[1]))) for data in train_data], [data[-1] for data in train_data]
test_x, test_y = [np.concatenate((get_word_expression_embedding(data[0]), get_word_expression_embedding(data[0]+";"+data[1]))) for data in test_data], [data[-1] for data in test_data]
svclassifier = train_svm(train_x, train_y, test_x, test_y)
y_pred = svclassifier.predict(test_x)
print(confusion_matrix(test_y,y_pred))  
print(classification_report(test_y,y_pred))

N is not found (biogas residue;test impact application various N containing organic substrate OS)
K is not found (NPK # nitrogen phosphoruspotassium;management practice MP;K application)
K is not found (NK # natural killer;balanced application;K application)
N is not found (soil N storage)
N is not found (soil N storage;soil water storage ; storage intervention)
P is not found (P truncatus # )
P is not found (P truncatus # ;P truncatus;storage intervention)
N is not found (tropical pasture;N isotope dilution technique)
L is not found (reduced level L dopa)
L is not found (reduced level L dopa;processing technique)
W is not found (programme;programme;programme;certification programme;programme;PBB programme;reintroduction programme;programme;W salutaris reintroduction programme;programme;major cocoa intervention programme;programme;redistributive land reform programme;programme;fast track land reform programme;programme;homestead food garden programme;programme;SMS programme;programme;a

/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warni

[0.76054001 0.68365907 0.70723898 0.71808305 0.67094814 0.67084935
 0.6862152  0.6894261  0.82469537 0.76044485]
Cross validation score(F1): 0.717210
Test accuracy : 0.750000
[[ 91   1   5   3   0  35]
 [  0  40   0   0   0   7]
 [  3   1  21   0   0   3]
 [  4   0   0  20   0   3]
 [  0   0   0   0  21   3]
 [ 26   2   2   2   0 107]]
              precision    recall  f1-score   support

           0       0.73      0.67      0.70       135
           1       0.91      0.85      0.88        47
           2       0.75      0.75      0.75        28
           3       0.80      0.74      0.77        27
           4       1.00      0.88      0.93        24
           5       0.68      0.77      0.72       139

   micro avg       0.75      0.75      0.75       400
   macro avg       0.81      0.78      0.79       400
weighted avg       0.76      0.75      0.75       400



In [21]:
train_x, train_y = [get_word_expression_embedding(data[0]) for data in train_data], [data[-1] for data in train_data]
test_x, test_y = [get_word_expression_embedding(data[0]) for data in test_data], [data[-1] for data in test_data]
svclassifier = train_svm(train_x, train_y, test_x, test_y)
y_pred = svclassifier.predict(test_x)
print(confusion_matrix(test_y,y_pred))  
print(classification_report(test_y,y_pred))

/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warni

[0.66896899 0.70016271 0.68699353 0.67177482 0.57250327 0.64002391
 0.45791393 0.54588504 0.71300007 0.81116551]
Cross validation score(F1): 0.646839
Test accuracy : 0.825000
[[113   0   2   4   1  15]
 [  2  44   0   0   0   1]
 [  4   0  22   1   0   1]
 [  1   0   0  23   0   3]
 [  0   0   0   0  24   0]
 [ 31   0   1   3   0 104]]
              precision    recall  f1-score   support

           0       0.75      0.84      0.79       135
           1       1.00      0.94      0.97        47
           2       0.88      0.79      0.83        28
           3       0.74      0.85      0.79        27
           4       0.96      1.00      0.98        24
           5       0.84      0.75      0.79       139

   micro avg       0.82      0.82      0.82       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.83      0.82      0.83       400



In [14]:
train_pred = inteventions_labels_model.predict_for_df(interventions_df)
test_pred = inteventions_labels_model.predict_for_df(test_interventions_df)

INFO:tensorflow:Writing example 0 of 2008


I0611 11:50:00.431395 140031605708608 base_bert_model.py:424] Writing example 0 of 2008


INFO:tensorflow:*** Example ***


I0611 11:50:00.433030 140031605708608 base_bert_model.py:400] *** Example ***


INFO:tensorflow:guid: 


...

I0611 11:50:14.901948 140031605708608 saver.py:1270] Restoring parameters from interventions_model_1/model.ckpt-94


INFO:tensorflow:Running local_init_op.


I0611 11:50:15.752493 140031605708608 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0611 11:50:15.845058 140031605708608 session_manager.py:493] Done running local_init_op.


In [22]:
svc_train_pred = svclassifier.predict_proba(train_x)
svc_test_pred = svclassifier.predict_proba(test_x)

In [23]:
ada_boost_x = np.concatenate([train_pred[0], svc_train_pred],axis=1)
ada_boost_test_x = np.concatenate([test_pred[0], svc_test_pred],axis=1)

In [24]:
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
boost = GradientBoostingClassifier(n_estimators  = 50, max_depth=1)
boost.fit(ada_boost_x, train_y)
print(boost.score(ada_boost_test_x, test_y))
print(boost.score(ada_boost_x, train_y))

scores = cross_validation(boost, np.concatenate((ada_boost_x,ada_boost_test_x),axis=0), train_y+test_y)
print(scores)
print("Cross validation score(F1): %f"%(scores.mean()))

0.8325
0.9825697211155379
[0.97416767 0.97545397 0.96636475 0.97370356 0.98386272 1.
 0.96877609 0.93511161 0.95004567 0.8233722 ]
Cross validation score(F1): 0.955086


In [25]:
adaboost_train = boost.predict(ada_boost_x)
adaboost_test = boost.predict(ada_boost_test_x)

In [26]:
print(confusion_matrix(test_y,adaboost_test))  
print(classification_report(test_y,adaboost_test))

[[120   0   3   3   1   8]
 [  2  45   0   0   0   0]
 [  6   0  21   0   0   1]
 [  3   0   0  21   0   3]
 [  0   0   0   0  24   0]
 [ 33   0   0   3   1 102]]
              precision    recall  f1-score   support

           0       0.73      0.89      0.80       135
           1       1.00      0.96      0.98        47
           2       0.88      0.75      0.81        28
           3       0.78      0.78      0.78        27
           4       0.92      1.00      0.96        24
           5       0.89      0.73      0.81       139

   micro avg       0.83      0.83      0.83       400
   macro avg       0.87      0.85      0.86       400
weighted avg       0.84      0.83      0.83       400



In [27]:
boost.feature_importances_

array([0.12875007, 0.00452847, 0.00171272, 0.02193731, 0.0041636 ,
       0.03223092, 0.17300299, 0.14202616, 0.09167584, 0.0567563 ,
       0.07749626, 0.26571935])

In [24]:
from bert_models import base_bert_model_interventions
base_bert_model_interventions = reload(base_bert_model_interventions)
inteventions_labels_model = base_bert_model_interventions.BaseBertModelInterventions("interventions_model_2", list(range(6)), gpu_device_num_hub=1, gpu_device_num =2, 
                                                         batch_size = 64, max_seq_length = 64, label_column="label")

Started tokenizer loading
Model partly loaded
Tokenizrer loaded
Config is done


In [25]:
from sklearn.utils import shuffle
interventions_df = shuffle(interventions_df)

In [26]:
inteventions_labels_model.train_model(interventions_df, interventions_df)

Beginning Training!


/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Training took time  0:00:59.565586
[[652   2   1   2   1  19]
 [  6 202   0   1   1  27]
 [ 15   1 114   0   1  11]
 [  2   0   0 122   0   2]
 [  2   2   0   0 120   0]
 [  7   6   3   0   2 676]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.96       677
           1       0.95      0.85      0.90       237
           2       0.97      0.80      0.88       142
           3       0.98      0.97      0.97       126
           4       0.96      0.97      0.96       124
           5       0.92      0.97      0.95       694

   micro avg       0.94      0.94      0.94      2000
   macro avg       0.95      0.92      0.94      2000
weighted avg       0.94      0.94      0.94      2000

F1 score:  0.9358171709079709


([array([0.05229361, 0.01952783, 0.00848623, 0.00901349, 0.00448894,
         0.9061899 ], dtype=float32),
  array([0.9075118 , 0.00489847, 0.01592019, 0.00443122, 0.00252644,
         0.06471194], dtype=float32),
  array([0.02205618, 0.5290204 , 0.18156017, 0.01430384, 0.05696338,
         0.19609602], dtype=float32),
  array([0.12718034, 0.07060011, 0.6467324 , 0.00922467, 0.0215151 ,
         0.12474736], dtype=float32),
  array([0.02730835, 0.05585728, 0.01050799, 0.00549356, 0.00369883,
         0.89713395], dtype=float32),
  array([0.0096434 , 0.5305489 , 0.3456425 , 0.01618847, 0.04350143,
         0.05447536], dtype=float32),
  array([0.02397994, 0.01092679, 0.01001079, 0.894642  , 0.03740696,
         0.02303355], dtype=float32),
  array([0.8612951 , 0.0106662 , 0.02007181, 0.03230768, 0.00744174,
         0.06821734], dtype=float32),
  array([0.7959355 , 0.01039973, 0.09092808, 0.00785386, 0.00714185,
         0.08774102], dtype=float32),
  array([0.03420011, 0.03710824, 0.01

In [27]:
inteventions_labels_model.evaluate_model(test_interventions_df)

[[92  3  9  1  0 30]
 [ 2 35  1  0  0  9]
 [ 9  0 18  0  0  1]
 [ 0  0  0 23  0  4]
 [ 6  2  0  2 14  0]
 [32  4  1  4  0 98]]
              precision    recall  f1-score   support

           0       0.65      0.68      0.67       135
           1       0.80      0.74      0.77        47
           2       0.62      0.64      0.63        28
           3       0.77      0.85      0.81        27
           4       1.00      0.58      0.74        24
           5       0.69      0.71      0.70       139

   micro avg       0.70      0.70      0.70       400
   macro avg       0.75      0.70      0.72       400
weighted avg       0.71      0.70      0.70       400

F1 score:  0.7181408215309695


([array([0.77385885, 0.00731763, 0.01072983, 0.01440036, 0.00460787,
         0.18908545], dtype=float32),
  array([0.9176632 , 0.00606267, 0.01564185, 0.00913732, 0.00350644,
         0.04798862], dtype=float32),
  array([0.92355675, 0.0049089 , 0.02236335, 0.00549911, 0.00433918,
         0.03933277], dtype=float32),
  array([0.9391089 , 0.00599499, 0.01755757, 0.00606791, 0.00335736,
         0.02791329], dtype=float32),
  array([0.9392664 , 0.00664633, 0.01941336, 0.00644553, 0.00320184,
         0.02502644], dtype=float32),
  array([0.8048865 , 0.02648338, 0.04387946, 0.00511895, 0.0037892 ,
         0.11584254], dtype=float32),
  array([0.9178611 , 0.00829609, 0.04083361, 0.00596925, 0.00456639,
         0.02247353], dtype=float32),
  array([0.1558724 , 0.01601584, 0.01006406, 0.00926264, 0.00620055,
         0.80258447], dtype=float32),
  array([0.9307043 , 0.00693183, 0.02491433, 0.00767526, 0.00491225,
         0.02486198], dtype=float32),
  array([0.06672908, 0.02294515, 0.00